In [1]:
from sympy import symbols, init_printing
import sympy
import sympy.physics.mechanics as me
from pydy.system import System
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.integrate import ode, odeint
from sympy.utilities.lambdify import lambdastr
import InputShaping as shaping
from scipy.optimize import fsolve
import numpy as np
from CDPMv2_animation import animate as ani
%matplotlib inline
init_printing(True)

In [42]:
# Constants
# If this is changed the angle solution will not work correctly
workspace_width = 2.8956
middle = workspace_width/2

# Seconds to run the Simulation
# This number will be multiplied by 30 to give a ratio of 30pts/s
runtime = 20

plate_width = 0.133
plate_height = 0.0665

rod_length = 0.5
rod_radius = 0.00635

mass_of_plate = 0.23
mass_of_rod = 0.07

cable_K = 100.0
cable_C = 10.0 # Get in damping ratio
rod_k = 100.0
rod_c = 5.0

inertia_of_plate = (plate_width**2 + plate_height**2) * (mass_of_plate/12.0)
inertia_of_rod = (mass_of_rod/12)*(3*rod_radius**2 + rod_length**2)

In [43]:
# Create the frames
A = me.ReferenceFrame('A')
# B = me.ReferenceFrame('B')
# C = me.ReferenceFrame('C')

# Create the symbols
# x, z, e, theta, beta = me.dynamicsymbols('x z e theta beta')
x_dot, z_dot, e_dot = me.dynamicsymbols('x_dot z_dot e_dot')
theta_dot, beta_dot = me.dynamicsymbols('theta_dot beta_dot')
H, a, b, M, m, g, k, t = sympy.symbols('H a b M m g k t')
Ip, Ir, c, r, p, kr, cr, D = sympy.symbols('Ip Ir c r p kr cr D')
L1, L2 = me.dynamicsymbols('L1 L2')
# L1, L2 = sympy.symbols('L1 L2')

# Orient the Beta frame
# B = A.orientnew('B', 'Axis', [beta, A.y])
# C = A.orientnew('C', 'Axis', [theta, A.y])
# B.set_ang_vel(A, beta_dot * A.y)
# C.set_ang_vel(A, theta_dot * A.y)

In [44]:
def s_curve(CurrTime, Begin, Amp, RiseTime, StartTime):
    """
    This was copied from Dr. Vaughan's Input shaping Library
    I edited it to allow for a beginning value.

    Function to generate an s-curve command

    Arguments:
      CurrTime : The current timestep or an array of times
      Amp : The magnitude of the s-curve (or final setpoint)
      RiseTime : The rise time of the curve
      StartTime : The time that the command should StartTime
      Begin : The beginnning value

    Returns :
      The command at the current timestep or an array representing the command
      over the times given (if CurrTime was an array)
    """

    Amp = Amp - Begin
    scurve = (2.0 * ((CurrTime - StartTime)/RiseTime)**2 *
    (CurrTime-StartTime >= 0) * (CurrTime-StartTime < RiseTime/2)
    +(-2.0 * ((CurrTime - StartTime)/RiseTime)**2 + 4.0 * ((CurrTime -
    StartTime)/RiseTime) - 1.0) * (CurrTime-StartTime >= RiseTime/2) *
    (CurrTime-StartTime < RiseTime) + 1.0 * (CurrTime-StartTime >= RiseTime))

    return (Amp * scurve) + Begin

In [45]:
for j in range(2):
    if j == 0:
        x, z, e, theta, beta = sympy.symbols('x z e theta beta')
        # Orient the Beta frame
        B = A.orientnew('B', 'Axis', [beta, A.y])
        C = A.orientnew('C', 'Axis', [theta, A.y])
        B.set_ang_vel(A, beta_dot * A.y)
        C.set_ang_vel(A, theta_dot * A.y)
    else:
        x, z, e, theta, beta = me.dynamicsymbols('x z e theta beta')
        # Orient the Beta frame
        B = A.orientnew('B', 'Axis', [beta, A.y])
        C = A.orientnew('C', 'Axis', [theta, A.y])
        B.set_ang_vel(A, beta_dot * A.y)
        C.set_ang_vel(A, theta_dot * A.y)
    # Create the origin points point
    A1 = me.Point('A1')
    A2 = me.Point('A2')

    # Set the origin points positions
    A1.set_pos(A1, 0)
    A2.set_pos(A1, H * A.x)

    # Create the plate center of Gravity
    G = me.Point('G')
    Gr = me.Point('Gr')

    # Set the COG position
    G.set_pos(A1, x*A.x + z*A.z)
    Gr.set_pos(G, e * C.z)

    # Create the attachment points
    B1 = me.Point('B1')
    B2 = me.Point('B2')

    # # Set the attachment points positions
    B1.set_pos(G, -a/2 * B.x - b/2 * B.z)
    B2.set_pos(G, a/2 * B.x - b/2 * B.z)

    # Create Rod top and Bottom points
    C1 = me.Point('C1')
    C2 = me.Point('C2')
    C1.set_pos(Gr, -D/2 * C.z)
    C2.set_pos(Gr, D/2 * C.z)

    # Create the position vectors
    a2 = A2.pos_from(A1)
    a2_x = a2 & A.x
    a2_z = a2 & A.z

    r1 = B1.pos_from(G)
    r1_x = r1 & A.x
    r1_z = r1 & A.z

    r2 = B2.pos_from(G)
    r2_x = r2 & A.x
    r2_z = r2 & A.z

    r21 = B2.pos_from(B1)
    r21_x = r21 & A.x
    r21_z = r21 & A.z

    s1 = B1.pos_from(A1)
    s2 = B2.pos_from(A2)
    
    spF1 = A1.pos_from(B1)
    spF2 = A2.pos_from(B2)

    # Calculating the Geometric lengths from the top corners of the plate
    Length1 = s1.magnitude()
    Length2 = s2.magnitude()

    # Creating the unit vectors pointing from the origins to the top plate points
    s1_vector = s1.normalize()
    s2_vector = s2.normalize()
    
    if j == 0:
        p = ((r21_x * x + a2_x * r1_x) * z - r21_z * x**2 +
             (r1_z * r2_x - r1_x * r2_z + a2_x * r21_z - a2_z * r2_x) * x +
             r1_x *  (a2_x * r2_z - a2_z * r2_x))
        equation_all = p.subs({H:workspace_width, a:plate_width,
                               b:plate_height}).evalf()
        equation_all = equation_all.simplify()
        lambda_str1 = lambdastr((beta), equation_all)
        lambda_str2 = lambda_str1.replace('sin', 'np.sin')
        lambda_str3 = lambda_str2.replace('cos', 'np.cos')
        lambda_str4 = lambda_str3.replace('x', 'x_temp')
        lambda_str = lambda_str4.replace('z', 'z_temp')
        func1 = eval(lambda_str)

In [46]:
# Set velocity of origin points
A1.set_vel(A, 0)
A2.set_vel(A, 0)

# Set velocity of COG
G.set_vel(A, x_dot * A.x + z_dot * A.z)
G.set_vel(B, 0)

Gr.set_vel(C, e_dot * C.z)
Gr.v1pt_theory(G, A, C)
# Gr.v2pt_theory(G, A, C)

# Set velocity of attachment points
B1.v2pt_theory(G, A, B)
B2.v2pt_theory(G, A, B)
B1.set_vel(B,0)
B2.set_vel(B,0)

In [47]:
# Set up the kinematic differential equations
kde = [x_dot - x.diff(t),
       z_dot - z.diff(t),
       e_dot - e.diff(t),
       beta_dot - beta.diff(t),
       theta_dot - theta.diff(t)]

# Create the plate inertial tensor
I_plate = me.inertia(A, 0, Ip, 0)
inertia_plate = (I_plate, G)

# Create the rod inertial tensor
I_rod = me.inertia(A, 0, Ir, 0)
inertia_rod = (I_rod, Gr)

# Create the Rigid Bodies
Plate = me.RigidBody('Plate', G, B, M, inertia_plate)
Rod = me.RigidBody('Rod', Gr, C, m, inertia_rod)

In [48]:
# These functions do not allow the springs or dampers to push
K1 = lambda LamLenK1: cable_K * (LamLenK1 >= L1)
K2 = lambda LamLenK2: cable_K * (LamLenK2 >= L2)
C1 = lambda LamLenC1: cable_C * (LamLenC1 >= L1)
C2 = lambda LamLenC2: cable_C * (LamLenC2 >= L2)

In [49]:
# Creating the forces acting on the body
grav_force_plate = (G, M * g * A.z)
grav_force_rod = (Gr, m * g * A.z)

# spring_force_B1 = (B1, s1_vector * K1(Length1) * (L1 - Length1))
# spring_force_B2 = (B2, s2_vector * K2(Length2) * (L2 - Length2))

spring_force_B1 = (B1, s1_vector * cable_K * (L1 - Length1))
spring_force_B2 = (B2, s2_vector * cable_K * (L2 - Length2))

######## Push ##########
# spring_force_B1 = (B1, s1_vector * cable_K * (L1 - Length1))
# spring_force_B2 = (B2, s2_vector * cable_K * (L2 - Length2))
#######################

spring_rod = (Gr, -kr * e * C.z)
spring_rod_on_plate = (G, kr * e * C.z)

damper_rod = (Gr, -cr * e_dot * C.z)
damper_rod_on_plate = (G, cr * e_dot * C.z)

B1_velocity = x_dot*A.x + z_dot*A.z - b*beta_dot/2*B.x + a*beta_dot/2*B.z
B2_velocity = x_dot*A.x + z_dot*A.z - b*beta_dot/2*B.x - a*beta_dot/2*B.z

B1_damping = (-C1(Length1) * B1_velocity & s1_vector) * s1_vector
B2_damping = (-C2(Length2) * B2_velocity & s2_vector) * s2_vector

######## Push ##########
# B1_damping = (-cable_C * B1_velocity & s1_vector) * s1_vector
# B2_damping = (-cable_C * B2_velocity & s2_vector) * s2_vector
#######################

damp_B1 = (B1, B1_damping)
damp_B2 = (B2, B2_damping)

B_frame_damp = (B, -0.006 * beta_dot * B.y)
C_frame_damp = (C, -0.001 * theta_dot * C.y)

C_frame_spring = (C, 0.1 * (beta - theta) * C.y)
C_frame_spring_on_B = (B, 0.1 * (theta - beta) * B.y)

# Not sure if damping on the cables is necessary
loads = [grav_force_plate,
         grav_force_rod,
         spring_force_B1,
         spring_force_B2,
         spring_rod,
         spring_rod_on_plate,
         damper_rod,
         damper_rod_on_plate,
#          damp_B1,
#          damp_B2,
         B_frame_damp,
         C_frame_damp,
         C_frame_spring,
         C_frame_spring_on_B]

In [50]:
# Setting up the coordinates, speeds, and creating KanesMethod
coordinates = [x, z, e, beta, theta]
speeds = [x_dot, z_dot, e_dot, beta_dot, theta_dot]
kane = me.KanesMethod(A, coordinates, speeds, kde)

# Creating Fr and Fr_star
fr, frstar = kane.kanes_equations(loads, [Plate, Rod])

# Creating the PyDy System
sys = System(kane)

In [51]:
## Initial Conditions
# If init_L1 and L2 are 0 they will be solved for
# If e_init is 0 it's equilibrium will be solved for
# You can put init_x = middle 
init_x = middle - middle/2
end_x = middle + middle/2

init_z = 1.5
end_z = 0.5

init_L1 = 0
end_L1 = 0

init_L2 = 0
end_L2 = 0

e_init = 0

init_beta = 0
end_beta = 0

init_theta = 0
end_theta = 0

risetime = 1
startime = 5

In [52]:
# Linkage Geometry
z_temp = init_z
x_temp = init_x
tau_initial_guess = 0.08
if init_x == middle:
    init_beta = 0
else:
    tau_solution = fsolve(func1, tau_initial_guess)
    init_beta = (tau_solution)[0]

z_temp = end_z
x_temp = end_x
if end_x == middle:
    end_beta = 0
else:
    tau_solution = fsolve(func1, tau_initial_guess)
    end_beta = (tau_solution)[0]
if init_theta == 0:
    init_theta = init_beta
print("Initial Beta: {}\nFinal Beta:    {}".format(np.rad2deg(init_beta),
                                                np.rad2deg(end_beta)))

Initial Beta: -22.593022408390638
Final Beta:    10.528797327822344


In [53]:
# Getting the geometric lengths for testing
Len1_init = Length1.subs({x:init_x,
                     z:init_z,
                     H:workspace_width,
                     a:plate_width,
                     b:plate_height, 
                     beta:init_beta})
Len2_init = Length2.subs({x:init_x,
                     z:init_z,
                     H:workspace_width,
                     a:plate_width,
                     b:plate_height, 
                     beta:init_beta})

Len1_end = Length1.subs({x:end_x,
                     z:end_z,
                     H:workspace_width,
                     a:plate_width,
                     b:plate_height, 
                     beta:end_beta})
Len2_end = Length2.subs({x:end_x,
                     z:end_z,
                     H:workspace_width,
                     a:plate_width,
                     b:plate_height, 
                     beta:end_beta})

if init_L1 == 0:
    init_L1 = Len1_init
if init_L2 == 0:
    init_L2 = Len2_init
if end_L1 == 0:
    end_L1 = Len1_end
if end_L2 == 0:
    end_L2 = Len2_end
if e_init == 0:
    e_init = (9.81 * mass_of_rod) / rod_k
print('These are the geometric lengths\n----------------\
        \nInitial Length 1: {}\nInitial Length 2: {}\nEnd Length 1:     {} \
      \nEnd Length 2:     {}\ne:                {}'.format(init_L1,init_L2,
                                                           end_L1, end_L2,
                                                           e_init))

These are the geometric lengths
----------------        
Initial Length 1: 1.59387075357335
Initial Length 2: 2.57569513029028
End Length 1:     2.15427657542010       
End Length 2:     0.805516068497493
e:                0.006867000000000001


In [54]:
def Lengths_solver(init_x, init_z, init_beta , end_x, end_z, end_beta):
    fake_length1, fake_length2 = sympy.symbols('fake_length1 fake_length2')
    Fsp1 = cable_K*(Len1_init - fake_length1)*spF1
    Fsp2 = cable_K*(Len2_init - fake_length2)*spF2

    Fsp1_x = Fsp1 & A.x
    Fsp1_z = Fsp1 & A.z

    Fsp2_x = Fsp2 & A.x
    Fsp2_z = Fsp2 & A.z

    X_side = Fsp1_x + Fsp2_x
    Z_side = Fsp1_z + Fsp2_z + ((mass_of_plate + mass_of_rod) * 9.81)

    X_init = X_side.subs({x:init_x,
                     z:init_z,
                     H:workspace_width,
                     a:plate_width,
                     b:plate_height, 
                     beta:init_beta})

    Z_init = Z_side.subs({x:init_x,
                     z:init_z,
                     H:workspace_width,
                     a:plate_width,
                     b:plate_height, 
                     beta:init_beta})

    Fsp1 = cable_K*(Len1_end - fake_length1)*spF1
    Fsp2 = cable_K*(Len2_end - fake_length2)*spF2

    Fsp1_x = Fsp1 & A.x
    Fsp1_z = Fsp1 & A.z

    Fsp2_x = Fsp2 & A.x
    Fsp2_z = Fsp2 & A.z

    X_side = Fsp1_x + Fsp2_x
    Z_side = Fsp1_z + Fsp2_z + ((mass_of_plate + mass_of_rod) * 9.81)

    X_end = X_side.subs({x:end_x,
                     z:end_z,
                     H:workspace_width,
                     a:plate_width,
                     b:plate_height, 
                     beta:end_beta})

    Z_end = Z_side.subs({x:end_x,
                     z:end_z,
                     H:workspace_width,
                     a:plate_width,
                     b:plate_height, 
                     beta:end_beta})
    Equ_init = [X_init, Z_init]
    temp_init= sympy.linsolve(Equ_init,fake_length1,fake_length2)
    subtract_L1_init, subtract_L2_init = next(iter(temp_init))

    Equ_end = [X_end,Z_end]
    temp_end = sympy.linsolve(Equ_end,fake_length1,fake_length2)
    subtract_L1_end, subtract_L2_end = next(iter(temp_end))

    delta_init_L1 = init_L1 - subtract_L1_init
    delta_init_L2 = init_L2 - subtract_L2_init
    delta_end_L1  = end_L1 - subtract_L1_end
    delta_end_L2  = end_L2 - subtract_L2_end

    actual_init_L1 = subtract_L1_init - delta_init_L1
    actual_init_L2 = subtract_L2_init - delta_init_L2
    actual_end_L1 = subtract_L1_end - delta_end_L1
    actual_end_L2 = subtract_L2_end - delta_end_L2

    print('These are the actual lengths\n----------------\
            \nInitial Length 1: {}\nInitial Length 2: {}\nEnd Length 1:     {} \
          \nEnd Length 2:     {}'.format(actual_init_L1,
                                         actual_init_L2,
                                         actual_end_L1,
                                         actual_end_L2))
    return actual_init_L1, actual_init_L2, actual_end_L1, actual_end_L2

In [55]:
actual_init_L1,actual_init_L2,actual_end_L1,actual_end_L2 = Lengths_solver(
    init_x, init_z, init_beta,end_x, end_z, end_beta)

These are the actual lengths
----------------            
Initial Length 1: 1.56329420521651
Initial Length 2: 2.56585133237500
End Length 1:     2.12358588462108           
End Length 2:     0.708527833693501


In [56]:
fake_length1, fake_length2 = sympy.symbols('fake_length1 fake_length2')

single_Len1_init = Length1.subs({
                     H:workspace_width,
                     a:plate_width,
                     b:plate_height})
single_Len2_init = Length2.subs({
                     H:workspace_width,
                     a:plate_width,
                     b:plate_height})

single_Fsp1 = cable_K*(single_Len1_init - fake_length1)*spF1
single_Fsp2 = cable_K*(single_Len2_init - fake_length2)*spF2

single_Fsp1_x = single_Fsp1 & A.x
single_Fsp1_z = single_Fsp1 & A.z

single_Fsp2_x = single_Fsp2 & A.x
single_Fsp2_z = single_Fsp2 & A.z

single_X_side = single_Fsp1_x + single_Fsp2_x
single_Z_side = single_Fsp1_z + single_Fsp2_z + ((mass_of_plate + mass_of_rod) * 9.81)

In [57]:
def single_Length_solver(xs, zs, betas):

    Len1_init = Length1.subs({x:xs,
                         z:zs,
                         H:workspace_width,
                         a:plate_width,
                         b:plate_height, 
                         beta:betas})
    Len2_init = Length2.subs({x:xs,
                         z:zs,
                         H:workspace_width,
                         a:plate_width,
                         b:plate_height, 
                         beta:betas})

    X_init = single_X_side.subs({x:xs,
                     z:zs,
                     H:workspace_width,
                     a:plate_width,
                     b:plate_height, 
                     beta:betas})

    Z_init = single_Z_side.subs({x:xs,
                     z:zs,
                     H:workspace_width,
                     a:plate_width,
                     b:plate_height, 
                     beta:betas})

#     Sympy way
    Equ_init = [X_init, Z_init]
    temp_init= sympy.linsolve(Equ_init,fake_length1,fake_length2)
    subtract_L1_init, subtract_L2_init = next(iter(temp_init))
    delta_init_L1 = Len1_init - subtract_L1_init
    delta_init_L2 = Len2_init - subtract_L2_init

    actual_init_L1 = subtract_L1_init - delta_init_L1
    actual_init_L2 = subtract_L2_init - delta_init_L2

    return actual_init_L1, actual_init_L2

In [58]:
X_Y_L1_L2_Beta = np.load("/Users/forrest/Desktop/test.npy")
# X_and_Y_points = 100

# x_workspace_min = 0.5
# x_workspace_max = 2.5

# y_workspace_min = 1
# y_workspace_max = 2

# Xp = np.linspace(x_workspace_min, x_workspace_max, num=X_and_Y_points)
# Yp = np.linspace(y_workspace_min, y_workspace_max, num=X_and_Y_points)
# n = X_and_Y_points

# dims = np.array((0,0))
# for i in range(n):
#     x_i = np.repeat(Xp[i],n)
#     total = np.column_stack((x_i,Yp))
#     dims = np.vstack((dims,total))
# dims = dims[1:]

# n = np.shape(dims)[0]
# Lengths_to_dims = np.array((0,0,0))
# for i in range(n):
#     z_temp = dims[i,1]
#     x_temp = dims[i,0]
#     tau_initial_guess = 0.08
#     tau_solution = fsolve(func1, tau_initial_guess)
#     init_beta = (tau_solution)[0]
#     Length1a, Length2a = single_Length_solver(dims[i,0], dims[i,1], init_beta)
#     Lengths_only_holder = np.column_stack((Length1a, Length2a, init_beta))
#     Lengths_to_dims = np.vstack((Lengths_to_dims, Lengths_only_holder))
#     print(i)
# Lengths_to_dims = Lengths_to_dims[1:]
# X_Y_L1_L2_Beta = np.hstack((dims, Lengths_to_dims))
# np.save('/Users/forrest/Desktop/test', X_Y_L1_L2_Beta)

In [59]:
# time = np.linspace(0.0, runtime, runtime * 30)
# straight_x = s_curve(time, init_x, end_x, risetime, startime)
# straight_z = s_curve(time, init_z, end_z, risetime, startime)
# straight_beta = np.empty((0, runtime * 30))
# for i in range (runtime * 30):
#     z_temp = straight_z[i]
#     x_temp = straight_x[i]
#     tau_initial_guess = 0.08
#     tau_solution = fsolve(func1, tau_initial_guess)
#     result = (tau_solution)[0]
#     straight_beta = np.append(straight_beta, result)

# straight_L1 = np.empty((0, runtime * 30))
# straight_L2 = np.empty((0, runtime * 30))
# for i in range (runtime * 30):
#     sL1, sL2 = single_Length_solver(straight_x[i], straight_z[i], straight_beta[i])
#     straight_L1 = np.append(straight_L1, sL1)
#     straight_L2 = np.append(straight_L2, sL2)

In [60]:
# straight_L1_f64 = straight_L1.astype('float64')
# straight_L2_f64 = straight_L2.astype('float64')

In [61]:
# from scipy.optimize import curve_fit
# def richard1(t,C,Q,B,v,M):
#     K = straight_L1_f64[-1]
#     A = straight_L1_f64[0]
#     w = A + ((K - A) / ((C + Q*np.e**(-B*(t - M)))**(1/v)))
#     return w

# def richard2(t,C,Q,B,v,M):
#     K = straight_L2_f64[-1]
#     A = straight_L2_f64[0]
#     w = A + ((K - A) / ((C + Q*np.e**(-B*(t - M)))**(1/v)))
#     return w
# popt1, pcov1 = curve_fit(richard1, time, straight_L1_f64)
# popt2, pcov2 = curve_fit(richard2, time, straight_L2_f64)
# print(popt1)
# print(popt2)
# plt.figure(figsize=(15,5))
# plt.subplot(1,2,1)
# plt.plot(time, straight_L1, label='data')
# plt.plot(time,richard1(time, *popt1), label='fit')
# plt.legend()
# plt.subplot(1,2,2)
# plt.plot(time, straight_L2, label='data')
# plt.plot(time,richard2(time, *popt2), label='fit')
# plt.legend()
# # y = richard(x, *popt)

In [ ]:
nat_freq_to_total = np.array((0,0,0,0))
damp_to_total = np.array((0,0,0,0))

# This will be done outside of the for loop to speed up computation

linearizer = kane.to_linearizer()
Maz, A, B = linearizer.linearize()

for i in range(n):

    op_point = {x:X_Y_L1_L2_Beta[i,0], z:X_Y_L1_L2_Beta[i,1],
            beta:X_Y_L1_L2_Beta[i,4], e:0.00686700,
            theta:X_Y_L1_L2_Beta[i,4],
        x_dot:0, z_dot:0, beta_dot:0, e_dot:0, theta_dot:0}

constants = {M: mass_of_plate,
             m: mass_of_rod,
             g: 9.81,
             H: workspace_width,
             a: plate_width,
             b: plate_height,
             kr: rod_k,
             cr: rod_c,
             Ip: inertia_of_plate,
             Ir: inertia_of_rod,
             L1:X_Y_L1_L2_Beta[i,2],
             L2:X_Y_L1_L2_Beta[i,3],
            }

    M_op = me.msubs(Maz, op_point)
    A_op = me.msubs(A, op_point)
    perm_mat = linearizer.perm_mat
    A_lin = perm_mat.T * M_op.LUsolve(A_op)
    A_lin_constants = me.msubs(A_lin, constants)
    A_sol = A_lin_constants.subs(op_point).doit()

    A_np = np.array(np.array(A_sol), np.float)

    eigenvals, eigenvects = np.linalg.eig(A_np)

    eigen = eigenvals[0:10:2]
    eigen_abs = np.abs(eigen)

    damp = np.abs(np.real(eigen)/eigen_abs)
    damp_index = np.argsort(damp)
    highd, middled, middled2, middled3, lowd = damp[damp_index][::-1][:5][0:5]
    # print('The fundamental damp is: {}'.format(lowd))
    # print('The second damp is: {}'.format(middled))
    # print('The third damp is: {}'.format(highd))

    eigen_index = np.argsort(eigen_abs)
    high, middle, middle2, middle3, low = eigen_abs[eigen_index][::-1][:5][0:5]
    # print('The fundamental frequency is: {}'.format(low))
    # print('The second frequency is: {}'.format(middle))
    # print('The third frequency is: {}'.format(high))

    print(i)

    nat_freq_columns = np.column_stack((low,middle,middle2,middle3,high))
    nat_freq_to_total = np.vstack((nat_freq_to_total, nat_freq_columns))

    damp_columns = np.column_stack((lowd,middled,middled2,middled3,highd))
    damp_to_total = np.vstack((damp_to_total, damp_columns))

nat_freq_to_total = nat_freq_to_total[1:]
damp_to_total = damp_to_total[1:]
XY_L1L2_natflm23h_damplm23h = np.hstack((X_Y_L1_L2_Beta,nat_freq_to_total,damp_to_total))